# Homework: Vector Search

# Q1. Getting the embeddings model

In [1]:
from sentence_transformers import SentenceTransformer


/home/roman/.local/share/virtualenvs/llm-zoomcamp-6WKyO3I_/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
model_name = "multi-qa-distilbert-cos-v1"
embedding_model = SentenceTransformer(model_name)

In [3]:
user_question = "I just discovered the course. Can I still join it?"

In [4]:
vector = embedding_model.encode(user_question)

In [5]:
vector[0]

0.078222655

What's the first value of the resulting vector?

-0.24 
-0.04 
**0.07**
0.27 


Answer: **0,07**

# Prepare the documents

In [6]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [7]:
len(documents)

948

In [8]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [9]:
documents = [x for x in documents if x['course'] == 'machine-learning-zoomcamp']

In [10]:
len(documents)

375

# Q2. Creating the embeddings

In [11]:
from tqdm import tqdm

In [12]:
embeddings = []
for x in tqdm(documents):
    text_qa = f"{x['question']} {x['text']}"
    embeddings.append(embedding_model.encode(text_qa))

100%|█████████████████████████████████████████████████████████████████████████████████| 375/375 [00:39<00:00,  9.48it/s]


In [13]:
import numpy as np

In [14]:
X = np.array(embeddings)

What's the shape of X? (X.shape). Include the parantheses.

In [15]:
X.shape

(375, 768)

# Q3. Search

In [16]:
scores = X.dot(vector)

In [17]:
max(scores)

0.6506574

What's the highest score in the results?

65.0  
6.5  
**0.65**  
0.065  

Answer: **0,65**

# Vector search

In [18]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(vector, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

# Q4. Hit-rate for our search engine

In [19]:
import pandas as pd

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [20]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [21]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [28]:
def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }
search_vector = lambda q: search_engine.search(embedding_model.encode(q['question']), num_results=5)

In [29]:
evaluate(ground_truth, search_vector)

100%|███████████████████████████████████████████████████████████████████████████████| 1830/1830 [02:44<00:00, 11.15it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

What did you get?

**0.93**  
0.73  
0.53  
0.33  

Answer: **0,93** 

# Q5. Indexing with Elasticsearch

In [31]:
len(vector)

768

In [40]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            # "question_vector": {
            #     "type": "dense_vector",
            #     "dims": 768,
            #     "index": True,
            #     "similarity": "cosine"
            # },
            # "text_vector": {
            #     "type": "dense_vector",
            #     "dims": 768,
            #     "index": True,
            #     "similarity": "cosine"
            # },
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [42]:
for i, doc in tqdm(enumerate(documents)):
    doc['question_text_vector'] = X[i]
    es_client.index(index=index_name, document=doc)

375it [00:04, 76.96it/s]


In [43]:
def elastic_search_knn(field, vector, course):
    knn = {
        "field": field,
        "query_vector": vector,
        "k": 5,
        "num_candidates": 10000,
        "filter": {
            "term": {
                "course": course
            }
        }
    }

    search_query = {
        "knn": knn,
        "_source": ["text", "section", "question", "course", "id"]
    }

    es_results = es_client.search(
        index=index_name,
        body=search_query
    )
    
    result_docs = []
    
    for hit in es_results['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

In [44]:
elastic_search_knn("question_text_vector", vector, "machine-learning-zoomcamp")

[{'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'},
 {'question': 'I just joined. What should I do next? How can I access course materials?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the vid

What's the ID of the document with the highest score?

Answer: **ee58a693**

# Q6. Hit-rate for Elasticsearch

In [46]:
def question_text_vector_knn(q):
    question = q['question']
    course = q['course']

    v_q = embedding_model.encode(question)

    return elastic_search_knn('question_text_vector', v_q, course)

evaluate(ground_truth, question_text_vector_knn)

100%|███████████████████████████████████████████████████████████████████████████████| 1830/1830 [01:39<00:00, 18.41it/s]


{'hit_rate': 0.9398907103825137, 'mrr': 0.8516484517304189}

What's hitrate for our dataset for Elastic?

**0.93**  
0.73  
0.53  
0.33  

Answer: **0,93**